In [2]:
from transformers import LlamaForCausalLM, AutoTokenizer
from diarizationlm import utils


In [3]:
import json
import os
import glob
from transformers import LlamaForCausalLM, AutoTokenizer
from diarizationlm import utils

# Configuration
INPUT_FOLDER = "diarizationlm/input_test"  # Change this to your input folder
OUTPUT_FOLDER = "diarizationlm/output_test"  # Change this to your output folder
WORDS_PER_CHUNK = 500  # Words per chunk
MODEL_NAME = "google/DiarizationLM-8b-Fisher-v2"

# Create output folder if it doesn't exist
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

In [5]:
import torch

# Check if CUDA is available
print(f"CUDA available: {torch.cuda.is_available()}")

# Check which GPU you're using
if torch.cuda.is_available():
    print(f"GPU device: {torch.cuda.get_device_name()}")
    print(f"GPU count: {torch.cuda.device_count()}")
    print(f"Current device: {torch.cuda.current_device()}")
    
    # Check memory usage
    print(f"GPU memory allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"GPU memory reserved: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
else:
    print("Running on CPU - this will be very slow!")

CUDA available: True
GPU device: NVIDIA H200 NVL
GPU count: 1
Current device: 0
GPU memory allocated: 29.95 GB
GPU memory reserved: 78.98 GB


In [5]:
import json
import os
import glob
import re
import torch
from transformers import AutoTokenizer, LlamaForCausalLM
from diarizationlm import utils  

def parse_word_level(json_data):
    """
    Convert the single-utterance JSON into perfect 1:1 word-level entries.
    """
    if "utterances" not in json_data or len(json_data["utterances"]) != 1:
        raise ValueError("JSON must contain exactly one utterance")

    utt = json_data["utterances"][0]
    words = utt["hyp_text"].split()
    speakers = utt["hyp_spk"].split()

    if len(words) != len(speakers):
        raise ValueError("Word/Speaker mismatch in original JSON")

    word_level = []
    for i, (w, spk) in enumerate(zip(words, speakers)):
        word_level.append({
            "word": w,
            "spk": spk,
            "pos": i,
        })

    return word_level

def chunk_word_level(words, max_words=500):
    """
    Split words into chunks, but preserve speaker continuity
    """
    chunks = []
    current_chunk = []
    current_word_count = 0
    
    for i, word_obj in enumerate(words):
        # Start new chunk if we exceed max words
        if current_word_count >= max_words:
            chunks.append(current_chunk)
            current_chunk = []
            current_word_count = 0
        
        current_chunk.append(word_obj)
        current_word_count += 1
    
    # Add final chunk
    if current_chunk:
        chunks.append(current_chunk)
    
    return chunks

def chunk_to_hypothesis_optimized(chunk):
    """
    Convert chunk to hypothesis with speaker labels ONLY on speaker changes
    Format: "<speaker:X> word1 word2 word3 <speaker:Y> word4 word5"
    """
    if not chunk:
        return ""
    
    parts = []
    current_speaker = None
    
    for word_obj in chunk:
        word = word_obj["word"]
        speaker = word_obj["spk"]
        
        # Add speaker tag only when speaker changes
        if speaker != current_speaker:
            parts.append(f"<speaker:{speaker}>")
            current_speaker = speaker
        
        parts.append(word)
    
    return " ".join(parts)

def process_chunk_with_diarizationlm(word_chunk, model, tokenizer):
    """Process a single chunk with optimized hypothesis format"""
    # Convert word chunk to OPTIMIZED hypothesis format
    hypothesis_chunk = chunk_to_hypothesis_optimized(word_chunk)
    chunk_word_count = len(word_chunk)
    print(f"Processing chunk of {chunk_word_count} words...")
    
    # Clear cache for better performance
    torch.cuda.empty_cache()
    
    # Tokenize input
    inputs = tokenizer([hypothesis_chunk + " --> "], return_tensors="pt").to("cuda")
    
    # Generate completion with optimized settings
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=min(800, int(inputs.input_ids.shape[1] * 1.2)),
            use_cache=True,
            do_sample=False,
            num_beams=1,
            temperature=1.0,
            pad_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.05
        )
    
    # Decode completion
    completion = tokenizer.batch_decode(outputs[:, inputs.input_ids.shape[1]:],
                                      skip_special_tokens=True)[0]
    
    # Transfer completion to hypothesis text
    transferred_completion = utils.transfer_llm_completion(completion, hypothesis_chunk)
    
    return transferred_completion

def parse_diarization_output_optimized(output_text, original_chunk):
    """
    Parse optimized DiarizationLM output and return speakers for each word
    Handles the speaker-change-only format with better debugging
    """
    print(f"    DEBUG: Original chunk speakers: {[w['spk'] for w in original_chunk[:10]]}...")  # First 10
    print(f"    DEBUG: DiarizationLM output: {output_text[:200]}...")
    
    # Pattern to match speaker tags and subsequent words
    pattern = r"<speaker:(\d+)>\s*([^<]*)"
    matches = re.findall(pattern, output_text)
    
    print(f"    DEBUG: Found {len(matches)} speaker segments in output")
    
    # If no matches found, try to extract just the speaker assignments
    if not matches:
        print("    DEBUG: No speaker segments found with pattern, trying fallback...")
        # Fallback: extract all speaker mentions
        speaker_pattern = r"<speaker:(\d+)>"
        speaker_matches = re.findall(speaker_pattern, output_text)
        print(f"    DEBUG: Found {len(speaker_matches)} speaker tags with fallback")
        
        if speaker_matches:
            # If we only have speaker tags, assume they apply to subsequent words
            current_speaker = speaker_matches[0] if speaker_matches else original_chunk[0]["spk"]
            updated_speakers = [current_speaker] * len(original_chunk)
            print(f"    DEBUG: Using single speaker '{current_speaker}' for all words")
            return updated_speakers
        else:
            # Last resort: return original speakers
            print("    DEBUG: No speaker tags found, using original speakers")
            return [word["spk"] for word in original_chunk]
    
    updated_speakers = []
    word_index = 0
    
    for i, (spk, text) in enumerate(matches):
        words_in_segment = text.strip().split()
        print(f"    DEBUG: Segment {i}: speaker={spk}, words={len(words_in_segment)}")
        
        # Assign this speaker to all words in this segment
        for _ in words_in_segment:
            if word_index < len(original_chunk):
                updated_speakers.append(spk)
                word_index += 1
    
    print(f"    DEBUG: Assigned speakers to {len(updated_speakers)} words")
    
    # Handle case where output has fewer words than input
    if len(updated_speakers) < len(original_chunk):
        print(f"    WARNING: Output has {len(updated_speakers)} speakers but chunk has {len(original_chunk)} words")
        # Fill remaining with last speaker or original speakers
        last_speaker = updated_speakers[-1] if updated_speakers else original_chunk[0]["spk"]
        fill_count = len(original_chunk) - len(updated_speakers)
        print(f"    DEBUG: Filling {fill_count} words with speaker '{last_speaker}'")
        updated_speakers.extend([last_speaker] * fill_count)
    
    # Compare first few speakers to see if changes are happening
    print(f"    DEBUG: First 10 original speakers: {[w['spk'] for w in original_chunk[:10]]}")
    print(f"    DEBUG: First 10 updated speakers: {updated_speakers[:10]}")
    
    return updated_speakers

def apply_chunk_speaker_updates(global_words, chunk, updated_speakers):
    """
    Update speakers in the global word list using the precise word positions.
    """
    if len(chunk) != len(updated_speakers):
        raise ValueError(f"Mismatch: chunk has {len(chunk)} words but diarization output has {len(updated_speakers)} speakers")

    changes = 0
    for word_obj, new_spk in zip(chunk, updated_speakers):
        if word_obj["spk"] != new_spk:
            word_obj["spk"] = new_spk
            changes += 1
    
    return changes  # Return the number of actual changes

def rebuild_json(global_words, original_json):
    """
    Rebuild JSON with updated speakers - handles the optimized format seamlessly
    """
    words = [w["word"] for w in global_words]
    speakers = [w["spk"] for w in global_words]

    if len(words) != len(speakers):
        raise ValueError("Final mismatch — should never happen")

    new_utt = original_json["utterances"][0].copy()
    new_utt["hyp_text"] = " ".join(words)
    new_utt["hyp_spk"] = " ".join(speakers)

    new_json = original_json.copy()
    new_json["utterances"] = [new_utt]

    return new_json

def process_single_json_file(json_file_path, model, tokenizer, output_folder):
    """Process a single JSON file using OPTIMIZED speaker-change-only approach"""
    print(f"\nProcessing: {os.path.basename(json_file_path)}")
    
    try:
        # Load JSON data
        with open(json_file_path, 'r') as f:
            json_data = json.load(f)
        
        # Step 1: Parse into word-level entries
        print("Step 1: Converting to word-level entries...")
        word_level_entries = parse_word_level(json_data)
        
        if not word_level_entries:
            print("ERROR: No word-level entries generated")
            return None
            
        print(f"Total: {len(word_level_entries)} words")
        
        # Show original speaker distribution
        original_speakers = [w["spk"] for w in word_level_entries]
        unique_speakers = set(original_speakers)
        print(f"Original speaker distribution: { {spk: original_speakers.count(spk) for spk in unique_speakers} }")
        
        # Step 2: Split words into chunks
        print("Step 2: Splitting words into chunks...")
        word_chunks = chunk_word_level(word_level_entries, max_words=500)
        print(f"Created {len(word_chunks)} chunks")
        
        # Step 3: Process each word chunk with DiarizationLM
        print("Step 3: Processing word chunks with DiarizationLM (optimized format)...")
        
        # Create a working copy of the global words
        global_words = word_level_entries.copy()
        total_changes = 0
        
        for chunk_idx, word_chunk in enumerate(word_chunks):
            print(f"  Processing chunk {chunk_idx + 1}/{len(word_chunks)} ({len(word_chunk)} words)")
            
            # Show chunk statistics
            unique_speakers = len(set(word["spk"] for word in word_chunk))
            print(f"    Unique speakers in chunk: {unique_speakers}")
            
            # Process chunk with DiarizationLM
            transferred_completion = process_chunk_with_diarizationlm(word_chunk, model, tokenizer)
            
            # Parse the updated speakers from the completion
            updated_speakers = parse_diarization_output_optimized(transferred_completion, word_chunk)
            
            # Apply speaker updates to global word list and get change count
            changes = apply_chunk_speaker_updates(global_words, word_chunk, updated_speakers)
            total_changes += changes
            print(f"    Updated {changes}/{len(word_chunk)} speaker assignments")
        
        print(f"TOTAL CHANGES: {total_changes}/{len(word_level_entries)} words updated")
        
        # Show final speaker distribution
        final_speakers = [w["spk"] for w in global_words]
        unique_final_speakers = set(final_speakers)
        print(f"Final speaker distribution: { {spk: final_speakers.count(spk) for spk in unique_final_speakers} }")
        
        # Step 4: Rebuild JSON with updated speakers
        print("Step 4: Rebuilding JSON with updated speakers...")
        updated_json = rebuild_json(global_words, json_data)
        
        # Step 5: Save result
        output_path = os.path.join(output_folder, os.path.basename(json_file_path))
        with open(output_path, 'w') as f:
            json.dump(updated_json, f, indent=2)
        
        print(f"COMPLETED: Saved to {output_path}")
        return updated_json
        
    except Exception as e:
        print(f"ERROR processing file {json_file_path}: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

def process_all_json_files(input_folder, output_folder):
    """Process all JSON files in the input folder with optimized approach"""
    # Load model with optimizations for H200
    print("Loading DiarizationLM model with H200 optimizations...")
    try:
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        model = LlamaForCausalLM.from_pretrained(
            MODEL_NAME,
            torch_dtype=torch.float16,  # Use half precision for H200
            device_map="auto"
        )
        print("Model loaded successfully with float16 precision")
    except Exception as e:
        print(f"ERROR loading model: {str(e)}")
        return
    
    # Find all JSON files in input folder
    json_files = glob.glob(os.path.join(input_folder, "*.json"))
    
    if not json_files:
        print(f"No JSON files found in {input_folder}")
        return
    
    print(f"Found {len(json_files)} JSON files to process")
    
    # Process each file
    successful_files = 0
    for json_file in json_files:
        try:
            result = process_single_json_file(json_file, model, tokenizer, output_folder)
            if result is not None:
                successful_files += 1
        except Exception as e:
            print(f"ERROR processing {json_file}: {str(e)}")
            continue
    
    print(f"\n{'='*80}")
    print("OPTIMIZED PIPELINE COMPLETED!")
    print(f"Successfully processed {successful_files}/{len(json_files)} files")
    print(f"Input folder: {input_folder}")
    print(f"Output folder: {output_folder}")
    print(f"{'='*80}")

# Run the optimized pipeline
if __name__ == "__main__":
    # Define your folders here
    INPUT_FOLDER = "diarizationlm/inputsjsons"
    OUTPUT_FOLDER = "diarizationlm/output_test"
    
    process_all_json_files(INPUT_FOLDER, OUTPUT_FOLDER)

Loading DiarizationLM model with H200 optimizations...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Model loaded successfully with float16 precision
Found 17 JSON files to process

Processing: SBC034.json
Step 1: Converting to word-level entries...
Total: 2178 words
Original speaker distribution: {'2': 408, '1': 1770}
Step 2: Splitting words into chunks...
Created 5 chunks
Step 3: Processing word chunks with DiarizationLM (optimized format)...
  Processing chunk 1/5 (500 words)
    Unique speakers in chunk: 2
Processing chunk of 500 words...
    DEBUG: Original chunk speakers: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']...
    DEBUG: DiarizationLM output: <speaker:1> Okay, it's time to go out now we're closed we're closed We are closed now. Please leave. These kids came in and I was like we're closing in a few minutes so we'll wait until you pick this ...
    DEBUG: Found 1 speaker segments in output
    DEBUG: Segment 0: speaker=1, words=500
    DEBUG: Assigned speakers to 500 words
    DEBUG: First 10 original speakers: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
    